<h2> Genomic Data Analysis: mtDNA Heteroplasmy </h2> 
 

<div class="alert alert-block alert-danger">
    Before you start running this notebook, make sure you are using a Dataproc runtime. To do so,
    <ul>
        <li>Click on the Jupyter icon on the right-hand side of the screen</li>
        <li>Inside Recommended environments, select Hail Genomics Analysis, which creates the computer type Dataproc Cluster</li>
        <li>Select reasonable defaults for CPU, RAM, disk and numbers of workers</li>
        <li>Click on Next</li>
    </ul>
</div>


<h3>  Objectives </h3>
<br>
The All of Us Research Program has released the short read whole genome sequencing (srWGS) SNP / Indel (“small”) variants dataset as a Hail VariantDataset (VDS) in 2025 to reduce the dataset size. Hail has limited functional support for the Hail VariantDataset format, so we recommend researchers subset the Hail VDS and convert it to other file formats, like Hail MatrixTable, PLINK or VCF files for downstream analysis.

In [ ]:
from datetime import datetime
import os

bucket = os.getenv('WORKSPACE_BUCKET')
bucket


<h3> Import Hail and Initialize Spark </h3>

<ul>
    <li>What is Hail?</li>
    <li>What is the workspace bucket?</li>
</ul>

In [ ]:
import hail as hl
hl.default_reference(new_default_reference="GRCh38")

# READ THE DOCS 
# https://hail.is/
# sets the default reference genome to the argument
# Hail is built to scale and has first-class support 
# for multi-dimensional structured data, like the 
# genomic data in a genome-wide association study (GWAS).

In [ ]:
# confirm reference contigs 
hl.get_reference('GRCh38').contigs

<h4> Read Hail VDS </h4>

We use the environment variable `WGS_VDS_PATH` to load the srWGS Hail VDS.



In [ ]:
vds_srwgs_path = os.getenv("WGS_VDS_PATH")
vds_srwgs_path

In [ ]:
vds = hl.vds.read_vds(vds_srwgs_path)

In [ ]:
vds.variant_data.aggregate_rows(
    hl.agg.counter(vds.variant_data.locus.contig)
)

# run to confirm if mtDNA is avaiable 
# likely need to call variants directly from data 
# no eggress allowed ?


<h2> Basics of the VDS </h2>

<h4> Basics of the VDS  </h4>

A VariantDataset is the Hail implementation of a data structure called the “scalable variant call representation” (SVCR). It has component tables of reference information and variant information.



<h2> Reference Data  </h2>
<h4>  Reference Data </h4>

The reference information contains the sparse matrix of reference blocks, which is keyed only by locus, and contains an END field which denotes the last position included in the current reference block.

We can use

<ul>
    <li> the function vds.reference_data to get the reference information from the VDS, a MatrixTable with only reference block data,
    <li> the function count() to get number of rows and columns in the reference block data,
    <li> and the function describe() to get all fields in the reference block data.
</ul>

<h4>  Further Reading </h4>

<ul>
    <li> Sparse Matrices: https://en.wikipedia.org/wiki/Sparse_matrix
</ul>


In [ ]:
vds.reference_data.count()

# Note output
# (2860284724, 414830)
# columns are pre-sample, therefore N=414830 samples 
# rows are 2860284724 ??

In [ ]:
vds.reference_data.describe()

<h2>  Variant Data </h2> 
<h4>  Variant Data </h4> 

The varaint information contains the sparse matrix of variant data. We can use

<ul>
<li> the function vds.variant_data to get the reference information from the VDS, a MatrixTable with only variant data,
<li> the function count() to get number of rows and columns in the variant data,
<li> and the function describe() to get all fields in the variant data. Please refer to the article How the All of Us genomic data are organized to get detailed information about the fields.
<ul>
This will mainly focus on manipulating the variant information.

In [ ]:
vds.variant_data.count()

In [ ]:
vds.variant_data.describe()

In [ ]:
test_intervals = ['chr1:100M-200M', 'chr16:29.001M-29.002M']

# change coordinate to the variant you are interested in
# test_intervals = ['chr13:32355250-3235525]
# my main interest is chrM 
# run test cell below 

In [ ]:
vds = hl.vds.filter_intervals(
    vds,
    [hl.parse_locus_interval(x,)
     for x in test_intervals])

In [ ]:
vds.variant_data.count()


<ul>
<li> filtering by chromosome intervals
</ul>

In [ ]:
test_interval_chrM = ['chrM:1-16569']
vds = hl.vds.filter_intervals(
    vds,
    [hl.parse_locus_interval(x,)
    for x in test_interval_chrM])

In [ ]:
vds.variant_data.count()
# why does the following section return 0 intervals, assume 0 is for intervals?
# this is a major bottle neck 
# will review paper methods 
# skipped for now \
# testing chr1 heteroplasmy with age instead 

<ul>
<li> filtering by chromosome
</ul>

In [ ]:
vds = hl.vds.filter_chromosomes(vds, keep= ["chr1", "chr16"])
# test if chrM works (likely will not)
# work arounds

In [ ]:
vds.variant_data.count()

<ul>
<li> restricting to a single chromosome
</ul>

In [ ]:
vds = hl.vds.filter_chromosomes(vds, keep= ["chr16"])

In [ ]:
vds.variant_data.count()
# expecting 446 variants for 414830 samples 


<h2> Convert to dense MatrixTable </h2>
<h4> Convert to dense MatrixTable </h4>

We use the function vds.to_dense_mt to create a single, dense MatrixTable from the split VariantDataset representation.

Before converting to a dense MatrixTable, we will tranform and remove some entries to meet the conventional format of a MatrixTable and other file formats like VCF and PLINK.

<ul>
    <li> Transform Local Allelic Depth (LAD) </li>
</ul>
    
The All of Us VDS uses Local Alleles (LA) which has information for reference alleles and all alternative alleles across all samples in the array. We will need to convert Local Alleles related fields, Local Allele Depth (LAD) and Local Genotypte (LGT) to the Global Allele format fields (AD and GT), which have information for the reference allele and alternative allele for each sample in the array. We will use two diferent functions to convert LAD to AD and LGT to GT.

<ul>
     <li> Convert LAD to AD using the function annotate_entries and vds.local_to_global
</ul>

In [ ]:
mt = vds.variant_data.annotate_entries(AD = hl.vds.local_to_global(vds.variant_data.LAD, 
                                                                   vds.variant_data.LA, 
                                                                   n_alleles=hl.len(vds.variant_data.alleles), 
                                                                   fill_value=0, number='R'))

In [ ]:
mt = mt.annotate_entries(GT = hl.vds.lgt_to_gt(mt.LGT, mt.LA))

<ul>
     <li> Densify the MatrixTable
     <li> This step is necessary before performing any dense MatrixTable-related computations, such as computing AC / AF / AN, variant QC, or converting to VCF / PLINK / BGEN files, etc.

</ul>

In [ ]:
mt = hl.vds.to_dense_mt(hl.vds.VariantDataset(vds.reference_data, mt))

<ul>
     <li> Create fields AC / AF / AN
     <li> There are no Allele Counts (AC) / Allele Frequency (AF) / Allele Number (AN) in the VDS. We will use the function agg.call_stats to get the AC / AF / AN.

</ul>

In [ ]:
mt = mt.annotate_rows(info = hl.agg.call_stats(mt.GT, mt.alleles))


<ul>
     <li> Remove unneccessary fields
     <li> We will use the function drop to drop fields
</ul>

In [ ]:
fields_to_drop_list = ['as_vets','as_vqsr','LAD', 'LGT', 'LA',
            'tranche_data', 'truth_sensitivity_snp_threshold', 
             'truth_sensitivity_indel_threshold','snp_vqslod_threshold','indel_vqslod_threshold']

In [ ]:
mt = mt.drop(*(f for f in fields_to_drop_list if f in mt.entry or f in mt.row or f in mt.col or f in mt.globals))

In [ ]:
mt.describe()

In [ ]:
# out_path = f'{bucket}/data/test_hail.mt'
# mt.write(out_path, overwrite = True)

<h2> Convert to VCF file </h2>
<ul>
    <li> The fields AC and AF include info about the reference alleles, which does not comply with the VCF format. We will use the function transmute_rows to remove the first value in the AC and AF fields. </li>
</ul>

In [ ]:
mt_vcf = mt.transmute_rows(info = mt.info.annotate(AF=mt.info.AF[1:], 
                                                              AC=mt.info.AC[1:]))

We use the function export_vcf to create a VCF file. We choose tabix as False so we don't create an index file in this step, as it takes a long time to create the index file using Hail. We will use GATK to create an index file to save time and cost.

The VCF header will not be filled in properly with the default arguments. We provide a vcf_header.txt file with the path below, which contains the correct header. We use the function get_vcf_metadata to use it for the metadata argument.

In [ ]:
vcf_header = "gs://fc-secure-ff68a895-e88d-426e-9ae4-b3802c51b53b/data/vcf_header.txt"
metadata = hl.get_vcf_metadata(vcf_header)

In [ ]:
out_vcf = f'{bucket}/data/test_vcf.vcf.bgz'
hl.export_vcf(mt_vcf, out_vcf, tabix = False, metadata=metadata)

### Working with CRAM
CRAM are compressed SAM formats and will most likely include mitochondrial reads that can be processed with mtSwirl. 


In [ ]:
# extracting mtDNA \
# mtSwril will produce 

In [ ]:
# genomic_location = os.getenv("CDR_STORAGE_PATH")
# genomic_location

We use Hail in some of the examples to load the All of Us genomic data, so let's import Hail and initialize Spark.

The All of Us genomic data are using hg38 as reference, so we set the default reference "GRCh38" while initializing Spark.

In [ ]:
# import hail as hl


In [ ]:
# hl.default_reference(new_default_reference = "GRCh38")

In [ ]:
! gsutil -u $GOOGLE_PROJECT cat gs://fc-aou-datasets-controlled/v8/wgs/cram/manifest.csv | wc -l

In [ ]:
# ! gsutil -u $GOOGLE_PROJECT cat gs://fc-aou-datasets-controlled/v8/wgs/short_read/snpindel/clinvar/bed/clinvar.ucsc.bed >> clin.bed


In [ ]:
# ! gsutil -u $GOOGLE_PROJECT ls gs://fc-aou-datasets-controlled/v8/wgs/short_read/snpindel/clinvar/vcf

In [ ]:
# ! gsutil -u $GOOGLE_PROJECT cat gs://fc-aou-datasets-controlled/v8/wgs/cram/manifest.csv | head -n 10

In [ ]:
# ! gsutil -u $GOOGLE_PROJECT ls gs://fc-aou-datasets-controlled/v8/wgs/short_read/snpindel/clinvar/vcf/


In [ ]:
# ! gsutil -u $GOOGLE_PROJECT cat gs://fc-aou-datasets-controlled/v8/wgs/short_read/snpindel/clinvar/vcf/0000020064.interval_list | grep -i chrM


In [ ]:
# ! gsutil -u $GOOGLE_PROJECT cat gs://fc-aou-datasets-controlled/v8/wgs/short_read/snpindel/clinvar/vcf/0000020311.vcf.bgz | zcat | grep chrM


In [ ]:
# ! gsutil -u $GOOGLE_PROJECT ls gs://fc-aou-datasets-controlled/v8/wgs/short_read/snpindel/exome/vcf

In [ ]:
# ! gsutil -u gs://fc-aou-datasets-controlled/v8/wgs/short_read/snpindel/exome/vcf/0000020016.vcf.bgz | zcat | grep chrM


### Short Read Whole Genome Sequencing 
####  srWGS



In [ ]:
# vds_srwgs_path = os.getenv("WGS_VDS_PATH")
# vds_srwgs_path

In [ ]:
# vds = hl.vds.read_vds(vds_srwgs_path)

A VDS has component tables of reference information and variant information.

The reference information contains the sparse matrix of reference blocks, which is keyed only by locus, and contains an END field which denotes the last position included in the current reference block.

We can use function vds.reference_data to get the reference information from the VDS, a MatrixTable with only reference block data, the function count() to get number of rows and columns in the reference block data, and the function describe() to get all fields in the reference block data.

In [ ]:
# vds.reference_data.count()

In [ ]:
# vds.reference_data.describe()

In [ ]:
# mt = vds.variant_data
# mt.count()


In [ ]:
# mt_1 = mt.head(1)
# mt_1.locus.contig.describe()

In [ ]:
# mt_1.aggregate_rows(
#     hl.agg.collect_as_set(mt_1.locus.contig)
# )


In [ ]:
# mt.filter_rows(mt.locus.contig == "MT").head(1).rows().show()


In [ ]:
# mtM = mt.filter_rows(mt.locus.contig == "M")
# mtM.count()


In [ ]:
# ! gsutil -u $GOOGLE_PROJECT ls gs://fc-aou-datasets-controlled/v7/wgs/** | grep -i mt


## Build the analysis cohort

In [ ]:
# !gsutil ls -r $WORKSPACE_BUCKET | grep person_22012003


In [ ]:
# import pandas as pd
# import os

# bucket = os.environ["WORKSPACE_BUCKET"]

# csv_path = (
#     f"{bucket}/bq_exports/kchewe@researchallofus.org/"
#     "20260123/person_22012003/person_22012003_000000000000.csv"
# )

# print("Reading:", csv_path)

# # Read only the person_id column
# df = pd.read_csv(csv_path, usecols=["person_id"])

# # Write one ID per line (no header)
# df["person_id"].astype(str).to_csv(
#     "analysis_ids.txt",
#     index=False,
#     header=False
# )

# # Copy to bucket root
# !gsutil cp analysis_ids.txt $WORKSPACE_BUCKET/analysis_ids.txt


In [ ]:
# !gsutil ls $WORKSPACE_BUCKET/analysis_ids.txt


In [ ]:
# import hail as hl

# ids = hl.import_table(
#     f"{os.environ['WORKSPACE_BUCKET']}/analysis_ids.txt",
#     no_header=True
# ).key_by('f0')

# ids.count()


In [ ]:
# mt = vds.variant_data
# mt = mt.filter_cols(hl.is_defined(ids[mt.s]))
# mt.count()


In [ ]:
# mt_1 = mt.head(1)
# mt_1.locus.contig.describe()

In [ ]:
# mt_1.aggregate_rows(
#     hl.agg.collect_as_set(mt_1.locus.contig)
# )


## Isolate chrM

In [ ]:
# # what contigs exist 
# mt.locus.contig.collect()


In [ ]:
mtM = mtM.annotate_entries(
    dp = hl.sum(mtM.AD)
)

mt_depth = mtM.group_cols_by(mtM.s).aggregate(
    mean_chrM_dp = hl.agg.mean(mtM.dp)
).cols()


In [ ]:
# type(mt)


In [ ]:
# mt.describe()


In [ ]:
# mt.locus.dtype


In [ ]:
# mt.aggregate_rows(
#     hl.agg.any(mt.locus.contig == "MT")
# )


In [ ]:
# mt.aggregate_rows(
#     hl.agg.take(mt.locus.contig, 20)
# )
